In [ ]:
import pandas as pd
import jieba

#分词以及去除停用词
def segmentor(source_file, stopwordslist):
    content = []
    stop_words = {}.fromkeys([line.rstrip() for line in (open(stopwordslist, encoding='utf8'))])
    stop_words = set(stop_words)
    data = pd.read_csv(source_file, encoding='utf8', sep='\t')
    text = data['text']
    label = data['label']
    for row in text:
        word_list = jieba.lcut(row)
        words = [word for word in word_list if word not in stop_words]
        sentence = ' '.join(words)
        content.append(sentence)
    return label, content

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn import metrics

# 先转换为词频矩阵形式，再转化为tfidf矩阵，最后转化为DMatrix形式

def convert2DMatrix(label, content):
    content = pd.DataFrame(content, columns=['text'])
    data = pd.concat([label, content], axis=1)
    c2n = lambda x : int(x)
    data['label'] = data['label'].apply(c2n)
    x_train, x_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.1)
    vectorizer = CountVectorizer(max_features=5000)
    tfidf_transformer = TfidfTransformer()
    tfidf = tfidf_transformer.fit_transform(vectorizer.fit_transform(x_train))
    x_train_weight = tfidf.to_array()
    tfidf = tfidf_transformer.transform(vectorizer.transform(x_test))
    x_test_weight = tfidf.to_array()
    dtrain = xgb.DMatrix(x_train_weight, label=y_train)
    dtest = xgb.DMatrix(x_test_weight, label=y_test)
    return dtrain, dtest
        

In [ ]:
import xgboost
    #xgboost进行分类
def xgboostClassifier(dtrain, dtest):
    # xgboost模型构建
    # 返回分类类别
    param = {'silent': 0, 'eta': 0.3, 'max_depth': 6, 'objective': 'multi:softmax', 'num_class': 3, 'eval_metric': 'merror'}  # 参数
    # 返回各类别概率
    param = {'silent': 0, 'eta': 0.3, 'max_depth': 6, 'objective': 'multi:softprob', 'num_class': 3, 'eval_metric': 'merror'}  # 参数
    evallist = [(dtrain, 'train'), (dtest, 'test')]
    num_round = 100  # 循环次数
    xgb_model = xgb.train(param, dtrain, num_round,evallist)
    # 保存训练模型
    # xgb_model.save_model('data/xgb_model')
    # xgb_model=xgb.Booster(model_file='data/xgb_model') #加载训练好的xgboost模型
        '''
    #利用训练完的模型直接测试
    xgb_model = xgb.Booster(model_file='data/xgb_model')  # init model #加载模型
    dtest = xgb.DMatrix('data/test.buffer')  #加载数据
    xgb_test(dtest,xgb_model)
    '''
 
    y_predict = xgb_model.predict(dtest)  # 模型预测
    label_all = ['负面', '中性','正面']
    confusion_mat = metrics.confusion_matrix(y_test, y_predict)
    df = pd.DataFrame(confusion_mat, columns=label_all)
    df.index = label_all
    print('准确率：', metrics.accuracy_score(y_test, y_predict))
    print('confusion_matrix:', df)
    print('分类报告:', metrics.classification_report(y_test, y_predict))

In [ ]:
# coding=utf-8
import pandas as pd
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.model_selection import train_test_split
 
if __name__ == '__main__':
    train_data = pd.read_csv('data/clean_data_train.csv', sep=',', names=['contents', 'labels']).astype(str)
    cw = lambda x: int(x)
    train_data['labels']=train_data['labels'].apply(cw)
 
    x_train, x_test, y_train, y_test = train_test_split(train_data['contents'], train_data['labels'], test_size=0.1)
 
    # 将语料转化为词袋向量，根据词袋向量统计TF-IDF
    vectorizer = CountVectorizer(max_features=5000)
    tf_idf_transformer = TfidfTransformer()
    tf_idf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(x_train))
    x_train_weight = tf_idf.toarray()  # 训练集TF-IDF权重矩阵
    tf_idf = tf_idf_transformer.transform(vectorizer.transform(x_test))
    x_test_weight = tf_idf.toarray()  # 测试集TF-IDF权重矩阵